In [1]:
import utils
import re
from typing import NamedTuple
import numpy as np

## Day 5: Supply Stacks

[#](https://adventofcode.com/2022/day/5) We have stacks of crates, and a giant crane which can move crates from one stack to the other one at a time. So we have the starting stack of crates, and list of instructions.


In [2]:
test: str = """    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2"""

inp = utils.get_input(5)

Since the input has a state and then the moves to do, seperating the two so its easier to parse:

In [5]:
state_str, moves_str = test.split("\n\n")
print(state_str)
print(moves_str)

    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 
move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2


I could just use a simple list to represent the move instructions, but this is a good place to use a named tuple so its clearer:

In [6]:
class Move(NamedTuple):
    crates: int
    From: int
    To: int


def parse_moves(moves: str = moves_str, debug=False):
    """takes a str of moves and returns a list of move tuples"""
    all_moves = []
    for move in moves.splitlines():
        nums = (int(n) for n in re.findall("[0-9]+", move))
        if debug:
            print(move)
            print(Move(*nums))
        all_moves.append(Move(*nums))
    return all_moves


parse_moves()

[Move(crates=1, From=2, To=1),
 Move(crates=3, From=1, To=3),
 Move(crates=2, From=2, To=1),
 Move(crates=1, From=1, To=2)]

The moves was easy, the tricky is how to represent the state. I'm going to use numpy here as that makes it easy to grab a column. Its def possible using just python, but would need to write a function to grab a col.

Numpy makes it easy to grab a column from a list of lists by using `[:, col_num]` on an array.

In [7]:
[[char for char in row] for row in state_str.splitlines()]

[[' ', ' ', ' ', ' ', '[', 'D', ']', ' ', ' ', ' ', ' '],
 ['[', 'N', ']', ' ', '[', 'C', ']', ' ', ' ', ' ', ' '],
 ['[', 'Z', ']', ' ', '[', 'M', ']', ' ', '[', 'P', ']'],
 [' ', '1', ' ', ' ', ' ', '2', ' ', ' ', ' ', '3', ' ']]

In [8]:
def parse_state(state: str = state_str, debug=False):
    stacks = {} 
    state = np.array([[char for char in row] for row in state.splitlines()])

    for i in range(1, len(state[0]), 4):
        tower = np.flip(np.array(state)[:, i])
        if debug:
            print(i, tower)

        # this is a bit hacky but hey it works
        stacks[int(tower[0])] = tower[np.where(tower != " ")][1:]

    return stacks


stacks = parse_state(debug=True)
stacks

1 ['1' 'Z' 'N' ' ']
5 ['2' 'M' 'C' 'D']
9 ['3' 'P' ' ' ' ']


{1: array(['Z', 'N'], dtype='<U1'),
 2: array(['M', 'C', 'D'], dtype='<U1'),
 3: array(['P'], dtype='<U1')}

All right! so now I can just pop a crate of the end of one stack and append to the other.

Putting the above together:

In [11]:
def parse(inp=test, verbose=False):
    """parses input""" 
    start_state, moves = inp.split("\n\n")
    return parse_state(start_state), parse_moves(moves)

parse()

({1: array(['Z', 'N'], dtype='<U1'),
  2: array(['M', 'C', 'D'], dtype='<U1'),
  3: array(['P'], dtype='<U1')},
 [Move(crates=1, From=2, To=1),
  Move(crates=3, From=1, To=3),
  Move(crates=2, From=2, To=1),
  Move(crates=1, From=1, To=2)])

In [16]:
def solve_1(inp=test, move_whole_stack=False):
    state, moves = parse(inp)

    for m in moves:
        if move_whole_stack:
            state[m.To] = np.append(state[m.To], state[m.From][-m.crates:])
        else:
            state[m.To] = np.append(state[m.To], state[m.From][-m.crates:][::-1])
        state[m.From] = state[m.From][:-m.crates]

    return "".join([state[key][-1] for key in sorted(state.keys())])

assert solve_1() == "CMZ"
solve_1(inp)

'HBTMTBSDC'

## Part 2

This is super simple - not instead of a move of multiple crates moving 1 crate at a time, the crane picks up the whole stack of crates and moves it.
So this is the same as above except for not reversing the pile of crates moved:



In [19]:
assert solve_1(move_whole_stack=True) == "MCD"
solve_1(inp, move_whole_stack=True)

'PQTJRSHWS'